## Import dependencies

In [96]:
import numpy as np
import pandas as pd

## Elliptic curve logic

### Logic for find order of concrete point

In [97]:
INFINITY = (0, 0)

In [98]:
def isPointValid(point):
    return point is not None and point is not np.nan

In [99]:
def arePointsEqual(p1, p2):
    return p1 == p2

In [100]:
def modulo_multiplicative_inverse(number, mod):
    # This will iterate from 0 to mod-1
    for i in range(0, mod):
        # If we have our multiplicative inverse then return it.
        if (number * i) % mod == 1:
            return i
    return -1

In [101]:
def calculate_s(numerator, denominator, mod):
    return (numerator * modulo_multiplicative_inverse(denominator, mod)) % mod

In [102]:
def sum(p1, p2, mod, a):
    if (not isPointValid(p1) or p1 == INFINITY): return p2
    if (not isPointValid(p2) or p2 == INFINITY): return p1
    s = 0
    if (arePointsEqual(p1, p2)):
        if (p1[1] == 0):
            return INFINITY
        numerator = 3*p1[0]*p1[0] + a
        denominator = 2*p1[1]
        s = calculate_s(numerator, denominator, mod)
    else:
        if (p1[0] == p2[0]):
            return INFINITY
        numerator = p2[1] - p1[1]
        denominator = p2[0] - p1[0]
        s = calculate_s(numerator, denominator, mod)    
        
    x = (s*s - p1[0] - p2[0]) % mod
    y = (s*(p1[0] - x) - p1[1]) % mod
    return (x, y)

In [103]:
def point_order(point, mod, a):
    i = 1
    result = point
    while (result != INFINITY):
        result = sum(point, result, mod, a)
        i = i + 1
    return i

### Logic for find all points

In [104]:
def get_modulas(mod):
    values = []
    for i in range(0, mod):
        values.append(i)
    return values

In [105]:
def get_y_square(a, b, mod):
    y_square = []
    for i in range(0, mod):
        result = (i**3 + a*i + b) % mod
        y_square.append(result)
    return y_square

In [106]:
def calculate_y_values(x, mod):
    y_value = None
    for i in range(0, mod // 2 + 1):
        r = i**2 % mod
        if (r == x):
            if (i == 0):
                y_value = str(i)
            else:
                y_value = "+-" + str(i)
    return y_value

In [107]:
def get_y_values(mod):
    y_values = []        
    for value in get_y_square(a, b, mod):
        y_values.append(calculate_y_values(value, mod))
    return y_values

In [108]:
def get_all_points_of_elliptic_curve(a, b, mod):
    df = pd.DataFrame(None, index=['x','y^2','y'], columns=np.arange(mod))
    df.loc['x'] = get_modulas(mod)
    df.loc['y^2'] = get_y_square(a, b, mod)
    df.loc['y'] = get_y_values(mod)
    return df

## Initialize params

### The elliptic curve is given in Weierstrass form
### y^2 = x^3 + a*x + b

### Get order of concrete point

In [109]:
point = (4, 2)
mod = 7
a = -1
point_order(point, mod, a)

4

### Get all points of elliptic curve

In [110]:
a = -1
b = 0
mod = 7
df_points = get_all_points_of_elliptic_curve(a, b, mod)
df_points

,0,1,2,3,4,5,6
x,0,1,2,3,4,5,6
y^2,0,0,6,3,4,1,0
y,0,0,None,None,+-2,+-1,0


### Count of all points of elliptic curve + INIFINITY point

In [111]:
count = 1 # Add INFINITY point
y_values = df_points.loc['y']
for y_value in y_values:
    if (y_value is None):
        continue    
    if ('+-' in y_value):
        count = count + 2
    else:
        count = count + 1
count

8